* https://huggingface.co/docs/bitsandbytes/v0.43.2/index
* https://huggingface.co/docs/peft/en/index

In [1]:
!pip install -qU bitsandbytes datasets peft trl

In [3]:
import json
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"

import warnings
warnings.filterwarnings(
    'ignore',
    message='The secret `HF_TOKEN` does not exist'
  )

import datasets
from datasets import Dataset
import huggingface_hub
import peft
import torch
from torch import cuda as torch_cuda
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
import trl

!pwd
import sys
sys.path.append("../llm-learning")

from utils import jupyter_formatting

jupyter_formatting.setup_notebook_formatting()

torch_cuda.is_available()

/content


True

In [4]:
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,
)

* https://huggingface.co/NousResearch/Meta-Llama-3.1-8B-Instruct

In [5]:
MODEL_ID = "NousResearch/Meta-Llama-3.1-8B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
if False:
    first_gpu = torch_cuda.get_device_properties(0)

    print(
        """\
        Memory allocated: {0:.2f} MB \
        Memory reserved: {1:.2f} MB\

        Total GPUs: {2}
        GPU's memory: {3:.1f} MB
        """.format(
                  torch_cuda.memory_allocated() / 1_024**2,
                  torch_cuda.memory_reserved() / 1_024**2,
                  torch_cuda.device_count(),
                  first_gpu.total_memory / 1_024**2,
              )
    )

In [7]:
if False:
    # Replace the model id with your specific model
    MODEL_ID = "NousResearch/Meta-Llama-3.1-8B-Instruct"

    # Download config.json file from the Hugging Face hub
    config_file_path = huggingface_hub.hf_hub_download(MODEL_ID, "config.json")
    print(config_file_path)

    index_file_path = huggingface_hub.hf_hub_download(
        MODEL_ID, "model.safetensors.index.json"
    )
    print(index_file_path)

In [8]:
if False:
    import json

    with open(config_file_path, "r") as config_file:
        config_data = json.load(config_file)
        print(json.dumps(config_data, indent=4))

    with open(index_file_path, "r") as index_file:
        index_data = json.load(index_file)
        print(json.dumps(index_data, indent=4))

In [9]:
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [10]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [11]:
!git clone https://github.com/lauramanor/legal_summarization

Cloning into 'legal_summarization'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 31 (delta 2), reused 0 (delta 0), pack-reused 25 (from 1)
Receiving objects: 100% (31/31), 136.60 KiB | 22.77 MiB/s, done.
Resolving deltas: 100% (10/10), done.


In [12]:
jsonl_array = []
with open("legal_summarization/tldrlegal_v1.json") as f:
    data = json.load(f)
    for _, value in data.items():
        jsonl_array.append(value)

In [13]:
jsonl_array[0].keys()

dict_keys(['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'])

* https://huggingface.co/docs/datasets/en/index

In [14]:
legal_dataset = Dataset.from_list(jsonl_array)

In [15]:
legal_dataset

Dataset({
    features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
    num_rows: 85
})

In [16]:
legal_dataset = legal_dataset.train_test_split(test_size=0.2)

In [17]:
legal_dataset_test_valid = legal_dataset["test"].train_test_split(test_size=0.5)

In [18]:
legal_dataset = datasets.DatasetDict(
    {
        "train": legal_dataset["train"],
        "test": legal_dataset_test_valid["test"],
        "validation": legal_dataset_test_valid["train"],
    }
)

In [19]:
legal_dataset

DatasetDict({
    train: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 68
    })
    test: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 9
    })
    validation: Dataset({
        features: ['doc', 'id', 'original_text', 'reference_summary', 'title', 'uid'],
        num_rows: 8
    })
})

In [20]:
legal_dataset["train"][0]

{
    "doc": "Android SDK License Agreement (June 2014)",
    "id": "543ed49a98d9d5a17e000268",
    "original_text": "you agree to use the sdk and write applications only for purposes that are
    permitted by a the license agreement and b any applicable law regulation or generally accepted
    practices or guidelines in the relevant jurisdictions including any laws regarding the export
    of data or software to and from the united states or other relevant countries.",
    "reference_summary": "stay within the law and license agreement.",
    "title": "4.2",
    "uid": "legalsum51"
}

### Instruction Templating

https://www.llama.com/docs/model-cards-and-prompt-formats/llama3_1/

In [21]:
INSTRUCTION_PROMPT_TEMPLATE = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please convert the following legal content into a short human-readable summary<|eot_id|><|start_header_id|>user<|end_header_id|>

[LEGAL_DOC]{LEGAL_TEXT}[END_LEGAL_DOC]<|eot_id|><|start_header_id|>assistant<|end_header_id|>"""

RESPONSE_TEMPLATE = """\
{NATURAL_LANGUAGE_SUMMARY}<|eot_id|><|end_of_text|>"""

In [22]:
def create_prompt(sample: str, include_response: bool = True) -> str:
    full_prompt = INSTRUCTION_PROMPT_TEMPLATE.format(LEGAL_TEXT=sample["original_text"])

    if include_response:
        full_prompt += RESPONSE_TEMPLATE.format(
            NATURAL_LANGUAGE_SUMMARY=sample["reference_summary"]
        )

    return full_prompt

In [23]:
print(create_prompt(legal_dataset["test"][1]))

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Please convert the following legal content into a short human-readable summary<|eot_id|><|start_header_id|>user<|end_header_id|>

[LEGAL_DOC]the app permits the purchase of virtual currency virtual money and use of that virtual money to purchase virtual items or services that we expressly make available for use in the app virtual goods. the purchase of virtual money and virtual goods is limited to account holders who are either a 18 years of age or older or b under the age of 18 and have the consent of a parent to make the purchase. parents of children under the age of 18 can consult the ios or google play settings for their app to restrict in app purchases but should also monitor their children s accounts for unexpected activity including the purchase of virtual money or virtual goods. purchases of virtual money and virtual goodsvirtual money is a category of content so the purchase of virtual money grants you only a limited

In [24]:
def generate_response(prompt, model, tokenizer):
    """
    Parameters:
      - prompt: str representing formatted prompt
      - model: model object
      - tokenizer: tokenizer object

    Functionality:
      This will allow our model to generate a response to a prompt!

    Returns:
      - str response of the model
    """

    # convert str input into tokenized input
    encoded_input = tokenizer(prompt, return_tensors="pt")

    # send the tokenized inputs to our GPU
    model_inputs = encoded_input.to("cuda")

    # generate response and set desired generation parameters
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # decode output from tokenized output to str output
    decoded_output = tokenizer.batch_decode(generated_ids)

    # return only the generated response (not the prompt) as output
    return decoded_output[0].split("<|end_header_id|>")[-1]

In [25]:
def generate_response(
    prompt: str,
    model: transformers.AutoModelForCausalLM,
    tokenizer: transformers.AutoTokenizer,
) -> str:
    # convert str input into tokenized input
    encoded_input = tokenizer(prompt, return_tensors="pt")

    # send the tokenized inputs to our GPU
    model_inputs = encoded_input.to("cuda")

    # generate response and set desired generation parameters
    generated_ids = model.generate(
        **model_inputs,
        max_new_tokens=256,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )

    # decode output from tokenized output to str output
    decoded_output = tokenizer.batch_decode(generated_ids)

    # return only the generated response (not the prompt) as output
    return decoded_output[0].split("<|end_header_id|>")[-1]

In [26]:
generate_response(
    create_prompt(legal_dataset["test"][1], include_response=False), model, tokenizer
)

"\n\nHere's a short human-readable summary:\n\n**Virtual Currency and Goods Terms**\n\nWhen you buy virtual currency and goods in our app, you're only getting a license to use them, not ownership. You can't sell, transfer, or exchange them for real money or goods. You can only obtain virtual currency and goods from us through our app or website, not from third-party platforms.\n\n**Key Points:**\n\n* You must be 18 or older, or have parental consent, to buy virtual currency and goods.\n* Virtual currency and goods have no monetary value and can't be sold or exchanged for real money or goods.\n* You can't trade or transfer virtual currency and goods to others unless we provide a feature to do so.\n* We can cancel any virtual currency or goods sold or transferred in violation of these terms.\n* Prices and availability of virtual currency and goods can change without notice.\n* All sales are final, and we won't permit exchanges or refunds.\n* If you're a parent, you're responsible for mon

In [27]:
# Ground Truth Summary
legal_dataset["test"][1]["reference_summary"]

'you can spend money on coins pokeballs. we ain t giving you that money back. want to buy gold online. too bad. oh and the eu gives people refund rights in some cases. not this one.'

In [28]:
# Let's try another just to see how the model responds to a different prompt.
generate_response(
    create_prompt(legal_dataset["test"][3], include_response=False), model, tokenizer
)

"\n\nHere's a short, human-readable summary:\n\n**Creating a YouTube Account**\n\nTo access some YouTube features, you'll need to create a YouTube or Google account. You're responsible for:\n\n* Providing accurate and complete information when creating your account\n* Keeping your account password secure\n* Notifying YouTube immediately if you suspect unauthorized use of your account\n\nIf someone uses your account without permission, you may be liable for any losses, but YouTube is not responsible for unauthorized use.<|eot_id|>"

In [29]:
# Ground Truth Summary
legal_dataset["test"][3]["reference_summary"]

'don t lie when you create your account. tell us immediately when your account has been compromised.'

### Required Post Processing

In [30]:
model_config = model.config
model = peft.prepare_model_for_kbit_training(model)

## Task #3: Setting up PEFT LoRA

In [31]:
def print_trainable_parameters(model: transformers.AutoModelForCausalLM) -> None:

    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        "trainable params: {0} || all params: {1} || trainable%: {2}".format(
            trainable_params, all_param, 100 * trainable_params / all_param
        )
    )

#### Initializing LoRA Config

https://huggingface.co/docs/peft/main/en/package_reference/lora#peft.LoraConfig
https://huggingface.co/docs/peft/main/en/package_reference/peft_model#peft.get_peft_model

In [32]:
# set our rank (higher value is more memory/better performance)
lora_r = 16

# set our dropout (default value)
lora_dropout = 0.1

# rule of thumb: alpha should be (lora_r * 2)
lora_alpha = 32

# construct our LoraConfig with the above hyperparameters
peft_config = peft.LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)

model = peft.get_peft_model(model, peft_config)

In [33]:
print_trainable_parameters(model)

trainable params: 41943040 || all params: 4582543360 || trainable%: 0.9152786281546499


In [34]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 4096)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): 

## Task #4: Training the Model


### Setting up Training

In [35]:
max_seq_length = 2_048

args = trl.SFTConfig(
    output_dir="llama381binstruct_summarize_short",
    max_steps=500,
    per_device_train_batch_size=1,
    warmup_steps=30,
    logging_steps=10,
    eval_strategy="steps",
    eval_steps=25,
    learning_rate=2e-4,
    lr_scheduler_type="linear",
    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": False,
    },
    max_seq_length=max_seq_length,
    packing=True,
)

* https://huggingface.co/docs/trl/en/sft_trainer

In [36]:
trainer = trl.SFTTrainer(
    model=model,
    peft_config=peft_config,
    tokenizer=tokenizer,
    formatting_func=create_prompt,
    args=args,
    train_dataset=legal_dataset["train"],
    eval_dataset=legal_dataset["validation"],
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:616: UserWarning: The passed formatting_func has more than one argument. Usually that function should have a single argument `example` which corresponds to the dictionary returned by each element of the dataset. Make sure you know what you are doing.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:421: UserWarning: You passed `packing=True` to the SFTTrainer/SFTConfig, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

## Task #5: Share Your Model!

In [ ]:
huggingface_hub.notebook_login()

In [ ]:
trainer.push_to_hub("SethWeidman/llama381binstruct_summarize_short_merged_test")

### Compare Outputs

In [ ]:
merged_model = model.merge_and_unload()

In [ ]:
merged_model.push_to_hub(
    "SethWeidman/llama381binstruct_summarize_short_merged_test",
    safe_serialization=True,
)

In [ ]:
tokenizer.push_to_hub("SethWeidman/llama381binstruct_summarize_short_merged_test")

In [ ]:
generate_response(
    create_prompt(legal_dataset["test"][1], include_response=False),
    merged_model,
    tokenizer,
)

In [ ]:
legal_dataset["test"][3]["original_text"]

In [ ]:
generate_response(
    create_prompt(legal_dataset["test"][3], include_response=False),
    merged_model,
    tokenizer,
)

In [ ]:
# Ground Truth Summary
legal_dataset["test"][3]["reference_summary"]